In [35]:
import numpy as np
import pandas as pd
import random as rand
import nashpy
from pulp import *
import matplotlib.pyplot as plt

### Задача 1

#### Ручной ввод

In [176]:
print("Введите матрицу весовых коэффициентов построчно!")
rows = list(input("Введите названия строк:").split())
columns = list(input("Введите названия столбцов:").split())
size = int(input("Введите количество строк:"))
matrix = pd.DataFrame(np.array([list(map(int, input(f'Введите {i+1} строку матрицы:\n').split())) for i in range(size)]), 
                        index = rows, columns = columns)
print('\nМатрица весовых коэффициентов:')
print(matrix)

Введите матрицу весовых коэффициентов построчно!
Введите названия строк:Семечки Гречка Тыква Рис Пшено
Введите названия столбцов:Пшено Овёс Гречка Рис Булгур
Введите количество строк:5
Введите 1 строку матрицы:
125 345 217 65 312
Введите 2 строку матрицы:
355 267 281 109 213
Введите 3 строку матрицы:
446 447 556 78 23
Введите 4 строку матрицы:
224 162 181 239 544
Введите 5 строку матрицы:
123 234 345 456 543

Матрица весовых коэффициентов:
         Пшено  Овёс  Гречка  Рис  Булгур
Семечки    125   345     217   65     312
Гречка     355   267     281  109     213
Тыква      446   447     556   78      23
Рис        224   162     181  239     544
Пшено      123   234     345  456     543


In [177]:
matrix

,Пшено,Овёс,Гречка,Рис,Булгур
Семечки,125,345,217,65,312
Гречка,355,267,281,109,213
Тыква,446,447,556,78,23
Рис,224,162,181,239,544
Пшено,123,234,345,456,543


#### Рандомная генерация

In [183]:
values = ['Телефоны', 'ПК', 'Ноутбуки', 'Наушники', 'Keyboards', 'CPUs']
si = rand.randint(1, 6)
index = []
cols = []
for i in range(si):
    ind_adding = values[rand.randint(0, len(values)-1)]
    cols_adding = values[rand.randint(0, len(values)-1)]
    if ind_adding in index:
        ind_adding += str(i)
    if cols_adding in index:
        cols_adding += str(i)
    index.append(ind_adding)
    cols.append(cols_adding)
matrix = pd.DataFrame(np.random.randint(0, 1000, (si, si)), index = index, columns=cols)
matrix

,ПК,CPUs,Телефоны
ПК,582,467,833
Ноутбуки,808,522,36
Keyboards,199,243,986


#### Считывание с csv

In [188]:
matrix = pd.read_csv('data8_1.csv', delimiter=';', encoding='utf-8', index_col=0)
matrix

,Телефоны,Наушники,Ноутбуки
Телефоны,300,280,230
Ноутбуки,200,180,130
ПК,240,380,400


In [189]:
np_matrix = matrix.values
lowest_game_price = np.max(np.min(np_matrix, axis=1))
print("Цена игры для игрока А при выборе чистой оптимальной стратегии: ", lowest_game_price)
lowest_game_price_tag = np.argmax(np.min(np_matrix, axis=1))
print("Оптимальная чистая стратегия для игрока А: ", matrix.index[lowest_game_price_tag])

Цена игры для игрока А при выборе чистой оптимальной стратегии:  240
Оптимальная чистая стратегия для игрока А:  ПК


In [190]:
highest_game_price = np.min(np.max(np_matrix, axis=0))
highest_game_price_tag = np.argmin(np.max(np_matrix, axis=0))
print("Цена игры для игрока B при выборе чистой оптимальной стратегии:", highest_game_price)
print("Оптимальная чистая стратегия для игрока А:", matrix.index[highest_game_price_tag])

Цена игры для игрока B при выборе чистой оптимальной стратегии: 300
Оптимальная чистая стратегия для игрока А: Телефоны


In [191]:
problem = LpProblem('Task8_1', LpMinimize)
x1 = LpVariable('x1', lowBound=0)
x2 = LpVariable('x2', lowBound=0)
x3 = LpVariable('x3', lowBound=0)

#Целевая функция и ограничения
problem += x1 + x2 + x3
problem += np_matrix[0][0]*x1 + np_matrix[1][0]*x2 + np_matrix[2][0]*x3 >= 1
problem += np_matrix[0][1]*x1 + np_matrix[1][1]*x2 + np_matrix[2][1]*x3 >= 1
problem += np_matrix[0][2]*x1 + np_matrix[1][2]*x2 + np_matrix[2][2]*x3 >= 1

problem.solve()
print("x1:", x1.varValue)
print("x2:", x2.varValue)
print("x3:", x3.varValue)
print("Целевая функция:", value(problem.objective))

x1: 0.0024691358
x2: 0.0
x3: 0.0010802469
Целевая функция: 0.0035493827


In [192]:
print("Цена игры: ", round(1/value(problem.objective),2))
columns1 = ["Стратегия P1", "Стратегия P2", "Стратегия P3"]
vals = [round((1/value(problem.objective)) * x1.varValue *100), 
        round((1/value(problem.objective)) * x2.varValue *100), 
        round((1/value(problem.objective)) * x3.varValue *100)]
res = pd.DataFrame(np.array(vals), index = columns1, columns = ["Проценты"])
res

Цена игры:  281.74


,Проценты
Стратегия P1,70
Стратегия P2,0
Стратегия P3,30


#### Итоги

In [193]:
print("Цена игры для игрока А при выборе чистой оптимальной стратегии: ", lowest_game_price)
print("Оптимальная чистая стратегия для игрока А: ", matrix.index[lowest_game_price_tag])
print("\nЦена игры для игрока B при выборе чистой оптимальной стратегии:", highest_game_price)
print("Оптимальная чистая стратегия для игрока А:", matrix.index[highest_game_price_tag])
print("\nТаблица смешанных стратегий для игрока А:")
print(res)
print("\nЦена игры: ", round(1/value(problem.objective)))

Цена игры для игрока А при выборе чистой оптимальной стратегии:  240
Оптимальная чистая стратегия для игрока А:  ПК

Цена игры для игрока B при выборе чистой оптимальной стратегии: 300
Оптимальная чистая стратегия для игрока А: Телефоны

Таблица смешанных стратегий для игрока А:
              Проценты
Стратегия P1        70
Стратегия P2         0
Стратегия P3        30

Цена игры:  282


### Задача 2

In [194]:
matrix2_a = pd.read_csv('data8_2_a.csv', delimiter=';', encoding='utf-8', index_col=0)
matrix2_a = matrix2_a.dropna(axis=0,how='all')
matrix2_a = matrix2_a.dropna(axis=1,how='all')
matrix2_a

,Телефоны,Ноутбуки,ПК
Телефоны,300.0,280.0,230.0
Ноутбуки,200.0,180.0,130.0
ПК,240.0,380.0,400.0


In [195]:
matrix2_b = pd.read_csv('data8_2_b.csv', delimiter=';', encoding='utf-8', index_col=0)
matrix2_b = matrix2_b.dropna(axis=0,how='all')
matrix2_b = matrix2_b.dropna(axis=1,how='all')
matrix2_b

,Телефоны,Наушники,Ноутбуки
Телефоны,180.0,110.0,240.0
Наушники,120.0,160.0,210.0
Ноутбуки,300.0,140.0,250.0


In [196]:
np_matrix2_a = matrix2_a.values
np_matrix2_b = matrix2_b.values
game_price_a = np.min(np.max(matrix2_a, axis=0))
game_price_a_tag = np.argmin(np.max(matrix2_a, axis=0))
print('Цена игры для игрока А при выборе чистой оптимальной стратегии :', game_price_a)
print('Оптимальная чистая стратегия для игрока А:', matrix2_a.index[game_price_a_tag])
game_price_b = np.min(np.max(matrix2_b, axis=1))
game_price_b_tag = np.argmin(np.max(matrix2_b, axis=1))
print('\nЦена игры для игрока B при выборе чистой оптимальной стратегии :', game_price_b)
print('Оптимальная чистая стратегия для игрока B:', matrix2_b.index[game_price_b_tag])
print('\nОбщая суммарная цена игры:', np.sum([game_price_b, game_price_a]))

Цена игры для игрока А при выборе чистой оптимальной стратегии : 300.0
Оптимальная чистая стратегия для игрока А: Телефоны

Цена игры для игрока А при выборе чистой оптимальной стратегии : 210.0
Оптимальная чистая стратегия для игрока А: Наушники

Общая суммарная цена игры: 510.0


In [197]:
balance_count = nashpy.Game(np_matrix2_a, np_matrix2_b)
balance = len(list(balance_count.support_enumeration()))
print('Количество равновесий по Нэшу:', balance)

Количество равновесий по Нэшу: 0


C:\Anaconda3\lib\site-packages\nashpy\algorithms\support_enumeration.py:196: RuntimeWarning: 
An even number of (0) equilibria was returned. This
indicates that the game is degenerate. Consider using another algorithm
to investigate.
                  
  warnings.warn(warning, RuntimeWarning)


In [198]:
problem2 = LpProblem("Task8_2", LpMinimize)

function_vars = [LpVariable('w1', lowBound=0), 
                 LpVariable('w2', lowBound=0), 
                 LpVariable('w3', lowBound=0)]
F = (np.array(function_vars)).sum()
problem2 += F

lims = np.vectorize(lambda x1, x2: x1 >= x2)((matrix2_b * function_vars).sum(axis=1), 1)
for c in lims:
    problem2 += c
problem2.solve()

a_strat = np.array(list(map(lambda x: x.value(), problem2.variables())))
a_strat = a_strat/a_strat.sum()
print ('Таблица смешанных стратегий для компании А:\n', a_strat)

Таблица смешанных стратегий для компании А:
 [0. 0. 1.]


In [199]:
problem2 = LpProblem("Task8_2", LpMaximize)

function_vars = [LpVariable('w1', lowBound=0), 
                 LpVariable('w2', lowBound=0), 
                 LpVariable('w3', lowBound=0)]
F = (np.array(function_vars)).sum()
problem2 += F

lims = np.vectorize(lambda x1, x2: x1 <= x2)((matrix2_a * function_vars).sum(axis=1), 1)
for c in lims:
    problem2 += c
problem2.solve()

b_strat = np.array(list(map(lambda x: x.value(), problem2.variables())))
b_strat = b_strat/b_strat.sum()
print ('Таблица смешанных стратегий для компании Б:\n', b_strat)

Таблица смешанных стратегий для компании Б:
 [0.73913043 0.         0.26086957]


In [200]:
a_game_price = ((a_strat * np_matrix2_a.T).T*b_strat).sum()
b_game_price = ((a_strat * np_matrix2_b.T).T*b_strat).sum()
print('Цена игры для игрока А при выборе смешанной оптимальной стратегии: ', a_game_price)
print('Цена игры для игрока Б при выборе смешанной оптимальной стратегии: ', b_game_price)
print('Общая цена игры в случае использования оптимальных стратегий: ', a_game_price + b_game_price)

Цена игры для игрока А при выборе смешанной оптимальной стратегии:  281.73913045438184
Цена игры для игрока Б при выборе смешанной оптимальной стратегии:  286.9565217330057
Общая цена игры в случае использования оптимальных стратегий:  568.6956521873875


#### Итоги

In [201]:
print('Цена игры для игрока А при выборе чистой оптимальной стратегии :', game_price_a)
print('Оптимальная чистая стратегия для игрока А:', matrix2_a.index[game_price_a_tag])
print('\nЦена игры для игрока B при выборе чистой оптимальной стратегии :', game_price_b)
print('Оптимальная чистая стратегия для игрока B:', matrix2_b.index[game_price_b_tag])
print('\nОбщая суммарная цена игры:', np.sum([game_price_b, game_price_a]))
print('\nКоличество равновесий по Нэшу:', balance)
print('\nТаблица смешанных стратегий для компании А:\n', a_strat)
print('Цена игры для игрока А при выборе смешанной оптимальной стратегии: ', a_game_price)
print('\nТаблица смешанных стратегий для компании Б:\n', b_strat)
print('Цена игры для игрока Б при выборе смешанной оптимальной стратегии: ', b_game_price)
print('\nОбщая цена игры в случае использования оптимальных стратегий: ', a_game_price + b_game_price)

Цена игры для игрока А при выборе чистой оптимальной стратегии : 300.0
Оптимальная чистая стратегия для игрока А: Телефоны

Цена игры для игрока А при выборе чистой оптимальной стратегии : 210.0
Оптимальная чистая стратегия для игрока А: Наушники

Общая суммарная цена игры: 510.0

Количество равновесий по Нэшу: 0

Таблица смешанных стратегий для компании А:
 [0. 0. 1.]
Цена игры для игрока А при выборе смешанной оптимальной стратегии:  281.73913045438184

Таблица смешанных стратегий для компании Б:
 [0.73913043 0.         0.26086957]
Цена игры для игрока Б при выборе смешанной оптимальной стратегии:  286.9565217330057

Общая цена игры в случае использования оптимальных стратегий:  568.6956521873875


### Задача 3

In [202]:
matrix3 = pd.read_csv('data8_3.csv', delimiter=';', encoding='utf-8', index_col=0)
matrix3 = matrix3.dropna(axis=0,how='all')
matrix3 = matrix3.dropna(axis=1,how='all')
matrix3

,Рецессия,Стагнация,Оживление,Подъем
Продукты,"50,00","30,00","35,00","45,00"
Косметика,"40,00","40,00","55,00","40,00"
Одежда,"35,00","40,00","45,00","55,00"
бытовая химия,"10,00","20,00","25,00","35,00"
Вероятности,"0,15","0,25","0,50","0,10"


In [203]:
np_chance_rate = matrix3.iloc[-1].values
np_chance_rate

array(['0,15', '0,25', '0,50', '0,10'], dtype=object)

In [204]:
np_matrix3 = matrix3.iloc[:-1].values
np_matrix3

array([['50,00', '30,00', '35,00', '45,00'],
       ['40,00', '40,00', '55,00', '40,00'],
       ['35,00', '40,00', '45,00', '55,00'],
       ['10,00', '20,00', '25,00', '35,00']], dtype=object)